# fakenews-pyspark-bert-train+infer

## Colab Setup

In [ ]:
! pip install -q pyspark==3.2.0 spark-nlp==3.4.2

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 142 kB 16.2 MB/s 
     |████████████████████████████████| 198 kB 49.7 MB/s 


In [ ]:
import sparknlp

spark = sparknlp.start(gpu = True, spark32=True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 3.4.2
Apache Spark version: 3.2.0


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


## Config

In [ ]:
class CFG:
    # Globals #
    EXP_ID = 'EXP_010' 
    seed = 111

In [ ]:
# Set Path
import os

ROOT =       '/content/drive/MyDrive/Projects/nishika/2022_fake_news_detection'
CUR_DIR =    ROOT + '/01_code/'
DATA_DIR =   ROOT + '/00_data/'
MODEL_DIR =  ROOT + '/02_model/'
LOG_DIR =    ROOT + f'/02_model/{CFG.EXP_ID}/log/'
RESULT_DIR = ROOT + '/03_result/'

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

%cd $CUR_DIR

/content/drive/MyDrive/Projects/nishika/2022_fake_news_detection/01_code


## Load Dataset

In [ ]:
trainDataset = spark.read.option("header", True).csv(DATA_DIR + 'train.csv')
holdout = spark.read.option("header", True).csv(DATA_DIR + 'test.csv')
sample = spark.read.option("header", True).csv(DATA_DIR + 'sample_submission.csv')

trainDataset.show(truncate=50)

+----------+------+-------------------------------------------------------------------------------------------------+
|        id|isFake|                                                                                             text|
+----------+------+-------------------------------------------------------------------------------------------------+
|d19828eb64|     1|       Cによると、アメリカの元大統領で、最長寿だったジョージ・ウォーカー・ブッシュ氏が27日(C-5...|
|dfaab096bd|     0|    中日新聞によると、コナミカップ・プロ野球アジアシリーズ2007の決勝戦・日本の中日ドラゴンズ対...|
|163504bf95|     1|      愛媛Cは、11月12日にリーグ準加盟の承認を受けて、来期リーグ加盟を目指す愛媛Cに対して、鈴木...|
|ed3c9dc579|     0|         国民日報によると3日、7時50分（UTC+9、日本時間と同じ）大韓民国京畿道平沢市の西海岸（ソ...|
|e06f88267f|     1|   共同通信によると、5日午後2時過ぎから東京都、神奈川県、千葉県の3都県の広い範囲の地域で停電が...|
|2f5903a788|     0|                2005年12月31日の河北新報、日刊スポーツによると、同年12月30日深夜10時35分ごろ、...|
|b2ca4f9386|     1|        日本経済新聞によると、日本バスケットボール協会は4月3日、2007年10月の開幕を目指す協会主...|
|ea52aa5790|     0|          神奈川新聞によると30日(UTC+9)、横浜市北部に同市営地下鉄の新線「グリーンライン（4号線...|
|a994696540|   

In [ ]:
from pyspark.sql.functions import col

print(f"Total {trainDataset.count()}")

trainDataset.groupBy("isFake") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

Total 3781
+------+-----+
|isFake|count|
+------+-----+
|     1| 1937|
|     0| 1844|
+------+-----+



In [ ]:
# set seed for reproducibility
(trainData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = CFG.seed)
print("Train Dataset Count: " + str(trainData.count()))
print("Test Dataset Count: " + str(testData.count()))

Train Dataset Count: 3021
Test Dataset Count: 760


## Model Pipeline

In [ ]:

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

word_segmenter = WordSegmenterModel.pretrained('wordseg_gsd_ud', 'ja')\
        .setInputCols(["document"])\
        .setOutputCol("token") 

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_iso", "ja")\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\

lemmatizer = LemmatizerModel.pretrained("lemma", "ja") \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("lemma")

# bert_embeddings = BertEmbeddings().pretrained(name="bert_base_japanese", lang="ja") \
#     .setInputCols(["document",'cleanTokens'])\
#     .setOutputCol("embeddings")

### bert pretrained 
# name='small_bert_L4_256', lang='en'
# name='sent_small_bert_L8_512', lang='en'
# name='bert_embeddings_bert_base_ja_cased', lang='ja'
# name="bert_base_japanese", lang="ja"


albert_embeddings = AlbertEmbeddings.pretrained("albert_embeddings_albert_base_japanese_v1","ja") \
    .setInputCols(["document", "lemma"]) \
    .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("isFake")\
    .setMaxEpochs(30)\
    .setLr(0.002)\
    .setDropout(0.5)\
    .setBatchSize(4)\
    .setEnableOutputLogs(True)\
    .setRandomSeed(CFG.seed)\
    .setOutputLogsPath(LOG_DIR)

# ClassifierDLApproach(Default): lr=0.005, batchSize=64, dropou=0.5, maxEpochs=30

bert_clf_pipeline = Pipeline(stages=[
    document_assembler,
    word_segmenter,
    normalizer,
    stopwords_cleaner,
    lemmatizer,
    albert_embeddings,
    embeddingsSentence,
    classsifierdl
])

wordseg_gsd_ud download started this may take some time.
Approximate size to download 979 KB
[OK!]
stopwords_iso download started this may take some time.
Approximate size to download 1.8 KB
[OK!]
lemma download started this may take some time.
Approximate size to download 3.4 MB
[OK!]
albert_embeddings_albert_base_japanese_v1 download started this may take some time.
Approximate size to download 43.5 MB
[OK!]


In [ ]:
# Transform 

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

transform_pipeline = Pipeline(stages=[
    document_assembler,
    word_segmenter,
    normalizer,
    stopwords_cleaner,
    lemmatizer, 
    finisher
])

transform_pipeline_run = transform_pipeline.fit(trainData)
transform_df = transform_pipeline_run.transform(trainData)

transform_df.show()

+----------+------+-------------------------------------+--------------------+----------------------+----------------------+----------------------+----------------------+------------------------------+
|        id|isFake|                                 text|            document|                 token|            normalized|           cleanTokens|                 lemma|                        tokens|
+----------+------+-------------------------------------+--------------------+----------------------+----------------------+----------------------+----------------------+------------------------------+
|0013fa1710|     1|      スポーツ仲裁裁判所(C)が2月11...|[{document, 0, 38...|[{token, 0, 3, スポ...|[{token, 0, 3, スポ...|[{token, 0, 3, スポ...|[{token, 0, 3, スポ...|  [スポーツ, 仲, 裁, 裁, 判...|
|0087cf99a4|     0|  現地時間の23日早朝、エジプト・シ...|[{document, 0, 53...|[{token, 0, 1, 現地...|[{token, 0, 1, 現地...|[{token, 0, 1, 現地...|[{token, 0, 1, 現地...|  [現地, 時間, 日, 早朝, エ...|
|00a34678bb|     1|朝日新聞、共同通信によると、みずほ...|[{docume

## Run

In [ ]:
# remove the existing logs

! rm -r {LOG_DIR}

In [ ]:
%%time
# training will take some time due to Bert (use GPU runtime when possible)

bert_clf_pipelineModel = bert_clf_pipeline.fit(trainData)

CPU times: user 3.32 s, sys: 565 ms, total: 3.88 s
Wall time: 9min 50s


In [ ]:
# Check log file

log_files = os.listdir(f'{LOG_DIR}')
log_files

['ClassifierDLApproach_45630e6891a9.log']

In [ ]:
# Read log file

log_file_name = os.listdir(f'{LOG_DIR}')[0]

with open(f'{LOG_DIR}{log_file_name}', "r") as log_file :
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 4 - training_examples: 3021 - classes: 2
Epoch 0/30 - 1.58s - loss: 497.48013 - acc: 0.59635764 - batches: 756
Epoch 1/30 - 1.29s - loss: 440.06073 - acc: 0.73112583 - batches: 756
Epoch 2/30 - 1.29s - loss: 425.20084 - acc: 0.7592715 - batches: 756
Epoch 3/30 - 1.30s - loss: 416.64523 - acc: 0.7705298 - batches: 756
Epoch 4/30 - 1.29s - loss: 410.4513 - acc: 0.7758278 - batches: 756
Epoch 5/30 - 1.30s - loss: 405.62384 - acc: 0.7834437 - batches: 756
Epoch 6/30 - 1.29s - loss: 401.70065 - acc: 0.789404 - batches: 756
Epoch 7/30 - 1.29s - loss: 398.4666 - acc: 0.79337746 - batches: 756
Epoch 8/30 - 1.31s - loss: 395.7562 - acc: 0.79701984 - batches: 756
Epoch 9/30 - 1.29s - loss: 393.39655 - acc: 0.79966885 - batches: 756
Epoch 10/30 - 1.29s - loss: 391.3382 - acc: 0.802649 - batches: 756
Epoch 11/30 - 1.29s - loss: 389.5192 - acc: 0.8076159 - batches: 756
Epoch 12/30 - 1.28s - loss: 387.89816 - acc: 0.8089404 - batches

In [ ]:
preds = bert_clf_pipelineModel.transform(testData)
preds_df = preds.select('isFake','text',"class.result").toPandas()

print(preds_df)

    isFake                                               text result
0        1  サウジアラビアのメッカで、将棋倒し事故があり、少なくとも717人が死亡、800人以上が負傷し...    [0]
1        1  エジプトのシナイ半島での爆弾テロ事件が、現地時間24日早朝2時に、エジプト・シナイ半島で起こ...    [1]
2        0  日刊スポーツによると、野球殿堂の2007年度の表彰者が1月12日発表された。それによると故・...    [0]
3        1  秋田魁新報と秋田放送によると、10日午前10時40分頃、東日本秋田支社から男性運転士(27歳...    [1]
4        1  欧州サッカー連盟のブラッター会長は26日、とスポンサーとしての権限を効かせイメージ悪化を抑止...    [1]
..     ...                                                ...    ...
755      0  12月26日午前9時6分頃[注釈 1]（UTC+9）、岐阜県関ケ原町内の東海道新幹線・米原駅...    [1]
756      1  産経新聞および読売新聞によると、東日本は6月27日、常磐線の広野−竜田(福島県広野町)間(5...    [1]
757      1  47(共同通信)・毎日新聞によると、5月17日に広島市中区の新ビジネスホテル「アソシア広島新...    [1]
758      1  朝日新聞によると、27日、鹿児島県鹿屋市の国道269号近くの歩道にあった郵便ポストがなくなっ...    [1]
759      1  毎日新聞・読売新聞によると、国際サッカー連盟・ジェローム・バルク事務局長(en)は13日、ス...    [0]

[760 rows x 3 columns]


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['result'], preds_df['isFake']))

              precision    recall  f1-score   support

           0       0.76      0.81      0.78       339
           1       0.84      0.79      0.81       421

    accuracy                           0.80       760
   macro avg       0.80      0.80      0.80       760
weighted avg       0.80      0.80      0.80       760



## Save Model

In [ ]:
# Save a Spark NLP pipeline
bert_clf_pipelineModel.save(f'{MODEL_DIR}{CFG.EXP_ID}/model')

In [ ]:
# # cd into saved dir and zip
# ! cd "{MODEL_DIR}{CFG.EXP_ID}/model" ; zip -r {MODEL_DIR}{CFG.EXP_ID}/my_nlp_pipeline.zip *

## Load Model

In [ ]:
# Setting (Colab Setup to Load Dataset)

loaded_bert_clf_pipelineModel = PipelineModel.load(f'{MODEL_DIR}{CFG.EXP_ID}/model')

## Infer

In [ ]:
# submission

preds = loaded_bert_clf_pipelineModel.transform(holdout)

preds_df = preds.select('id', 'text',"class.result").toPandas()

print(preds_df)

              id                                               text result
0     d253d7b7ac  共同通信によると、ペルーの政党「シ・クンプレ」が、6日、中央選管である全国選挙評議会 (JN...    [1]
1     fcfe44d0a0  自殺した川田亜子さんの元恋人とされる、映画監督で平和活動家のマット・テイラー氏が、川田さんが...    [1]
2     213caf5cf5  沖縄タイムスによると、アメリカ軍の海兵隊・二等軍曹(38歳)が2月10日深夜、沖縄市の公園近...    [1]
3     15aefc8374  動画投稿サイト「C2」に人気ドラマやバラエティ番組を投稿したとして、著作権法違反(公衆送信権...    [1]
4     aded40e220  報道機関各社によると、12月1日、東京都内で日本プロ野球の2次ドラフトが初めて行われた。サン...    [1]
...          ...                                                ...    ...
3776  f87ff9f55f  スポーツニッポンによると、2015年9月12日に投票が行われることになっている(欧州サッカー...    [1]
3777  f7bb120e80  朝日新聞・時事通信によると、大韓民国の仁川空港とソウル駅とを結ぶ空港鉄道（KORAIL空港鉄...    [0]
3778  09cb71299e  気象庁によると、台風17号「ルンビア」(umbia)は、7日午前10時現在、南鳥島の南西約2...    [1]
3779  ce8e47d692  中日新聞、朝日新聞、読売新聞、産経新聞、C、日本放送協会によると、インドネシアは27日、バン...    [1]
3780  044c06ac12  25日、高校の一部科目の履修漏れが判明した問題で、26日までに35都道県の254校(内、私立...    [1]

[3781 rows x 3 columns]


In [ ]:
# create submission_df
submission_df = preds_df[['id', 'result']].copy()

submission_df['result'] = submission_df['result'].str[0]
submission_df = submission_df.rename(columns={"result": "isFake"})

print(len(submission_df)) # Recheck 3781

submission_df

3781


,id,isFake
0,d253d7b7ac,1
1,fcfe44d0a0,1
2,213caf5cf5,1
3,15aefc8374,1
4,aded40e220,1
...,...,...
3776,f87ff9f55f,1
3777,f7bb120e80,0
3778,09cb71299e,1
3779,ce8e47d692,1


In [ ]:
# Save file

SAVE_DIR = RESULT_DIR + f'submission_{CFG.EXP_ID}.csv'
submission_df.to_csv(SAVE_DIR, index=False)

print(SAVE_DIR)

/content/drive/MyDrive/Projects/nishika/2022_fake_news_detection/03_result/submission_EXP_010.csv
